# Code for cropping ROI from RhizoVisionAnalyzer text file

In [ ]:
## if missing cv2 type "pip install opencv-python"
import cv2
import numpy as np
import glob
import csv
import os
from shutil import copyfile

#### [User Setup] define & set the working directory containing images and their annotations. Annotation files should have exact same name as image with a .txt extension. Change default ext if required.

In [ ]:
## Choose working directory with data, use forward slash (/) or double back slash (\\) in path
## macOS   '/Users/marcusgriffiths/Desktop/Folder'
## windows 'C:/Users/marcu/OneDrive/Desktop/Folder'
basedir = '/Users/marcusgriffiths/Desktop/Folder'
default_ext = '*.jpg'

#### [End User Setup] make "output" folder in basedir

In [ ]:
os.chdir(basedir)

outputdir = ("Cropped")
CHECK_FOLDER = os.path.isdir(outputdir)
# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(outputdir)
    print("created folder : ", outputdir)
else:
    print(outputdir, "folder already exists.")

noannotationdir = ("No_annotation")
CHECK_FOLDER = os.path.isdir(noannotationdir)
# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(noannotationdir)
    print("created folder : ", noannotationdir)
else:
    print(noannotationdir, "folder already exists.")

##### Define function to get ROI from txt file

In [ ]:
def getROIList(filename):
    result = []
    
    if os.path.exists(filename) == False:
        print(filename + ' has no annotation file')
        copyfile(filename[0:-3] + default_ext[2:], noannotationdir + '/' + filename[0:-3] + default_ext[2:])
        return result
    
    with open(filename, newline='') as csvfile:
        filereader = csv.reader(csvfile, delimiter=',')
        linecnt = 0
        
        for row in filereader:
            if linecnt == 0:
                linecnt += 1
                continue
            else:
                if not (int(row[4]) == 0 or int(row[5]) == 0):
                    result.append((int(row[2]),int(row[3]),int(row[4]),int(row[5])))
                linecnt += 1
    
    if len(result) == 0:
        print(filename + ' annotation is empty')
        copyfile(filename[0:-3] + default_ext[2:], noannotationdir + '/' + filename[0:-3] + default_ext[2:])
    
    return result

###### Define image save function. Use cv2.IMREAD_ANYCOLOR for color images, grayscale is used for root flatbed scans 

In [ ]:
def saveimages(roilist, imagename):
    img = cv2.imread(imagename, cv2.IMREAD_GRAYSCALE)
    idx = 1
    
    for roi in roilist:
        roiimg = img[roi[1] : roi[1] + roi[3], roi[0] : roi[0] + roi[2]]
        cv2.imwrite(outputdir + '/' + imagename[:-4] + '_r' + str(idx) + default_ext[1:], roiimg)
        idx += 1

###### Call functions to crop image based on coordinates & save output

In [ ]:
print('Working in ' + basedir)
#files = glob.glob('*.jpg')
files = glob.glob(default_ext)

for f in files:
    annfile = f[0:len(f)-(len(default_ext) - 2)] + 'txt'
    roilist = getROIList(annfile)
    saveimages(roilist, f)
print('done!')